The aim of this notebook is to make use of the word2vec model to find similar songs

In [1]:
import pandas as pd
import numpy as np
import gensim.models.word2vec as w2v
import multiprocessing
import os
import re
import pprint
import sklearn.manifold
import matplotlib.pyplot as plt

unable to import 'smart_open.gcs', disabling that module


In [2]:
!pip install smart_open.gcs

  ERROR: Could not find a version that satisfies the requirement smart_open.gcs (from versions: none)
ERROR: No matching distribution found for smart_open.gcs


Though non english artists were removed, the dataset contained Hindi lyrics of Lata Mangeshkar written in English. Therefore, I decided to remove all songs sung by her.

In [3]:
songs = pd.read_csv("hhgroups_merge_28_05.csv", header=0, encoding='utf-8')
#songs.head()
songs = songs[songs.artista != 'Lata Mangeshkar']
songs.head()

,id,artista,cancion,album,letra,anyo,visitas
0,0,Denom,Machete (con Jarfaiter y Gente jodida),Medicina,"Para su nuevo disco ""Medicina"", Denom ha vuelt...",2019,126
1,1,Denom,Vacío (con Ivo Incuerdo),Medicina,"[Denom]\nYo que quería, yo que pedía vida,\nSe...",2019,361
2,2,Denom,El orgullo es fiel (con Juancho Marqués y Elio...,Medicina,"""El orgullo es fiel"" es uno de los cortes incl...",2019,262
3,3,Denom,Mueve mueve (con Fernandocosta),Medicina,"[Estribillo: Denom] (x2)\nMueve, mueve, mueve,...",2019,578
4,4,Jaro Desperdizio,Insomnia,"Sin álbum, es un vídeo suelto","[Estribillo]\nY en esta noche, ¿Quién me arrop...",2019,219


To train the word2vec model, we first need to build its vocabulary. To do that, I iterated over each song and added it to an array that can later be fed to the model.

In [4]:
text_corpus = []
for song in songs['letra']:
    words = song.lower().split()
    text_corpus.append(words)


# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 50
# Minimum word count threshold.
min_word_count = 1

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7


downsampling = 1e-1

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

songs2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

songs2vec.build_vocab(text_corpus)
print (len(text_corpus))

9325


In [5]:
import time
start_time = time.time()



songs2vec.train(text_corpus, total_examples=songs2vec.corpus_count, epochs=2)

if not os.path.exists("trained"):
    os.makedirs("trained")

songs2vec.save(os.path.join("trained", "songs2vectors.w2v"))

print("--- %s seconds ---" % (time.time() - start_time))

--- 41.590837717056274 seconds ---


In [6]:
songs2vec = w2v.Word2Vec.load(os.path.join("trained", "songs2vectors.w2v"))

#### Let's explore our model

Find similar words

In [7]:
songs2vec.wv.most_similar("viajar")

[('volverme', 0.9707964658737183),
 ('echarme', 0.9676481485366821),
 ('descansar', 0.9658570885658264),
 ('pisar', 0.9652892351150513),
 ('quedarte', 0.9639395475387573),
 ('conquistar', 0.9639204740524292),
 ('marcharme', 0.9638046622276306),
 ('condenado', 0.9630315899848938),
 ('sonreir,', 0.9623922109603882),
 ('recorrer', 0.9617524147033691)]

In [8]:
songs2vec.wv.most_similar("vacaciones")

[('rama,', 0.9695250988006592),
 ('nadando', 0.9677631258964539),
 ('leña', 0.967171311378479),
 ('catorce', 0.9668575525283813),
 ('conociendo', 0.9649705290794373),
 ('pasear', 0.9645663499832153),
 ('círculos', 0.9642952680587769),
 ('diciembre,', 0.9642901420593262),
 ('currando', 0.9641720652580261),
 ('cargando', 0.9641454815864563)]

Words out of context

In [9]:
songs2vec.wv.doesnt_match("te quiero".split())

C:\Users\merxe\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'quiero'

In [10]:
songs2vec.most_similar(positive=['mujer', 'rey'], negative=['hombre'])


C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('fallaste,', 0.853793740272522),
 ('gorra', 0.8486242294311523),
 ('cerda', 0.8460404872894287),
 ('(ser', 0.8414132595062256),
 ('copa,', 0.8385828733444214),
 ('pose', 0.8353822231292725),
 ('maría', 0.8334468603134155),
 ('hija', 0.832923412322998),
 ('fiera,', 0.8325326442718506),
 ('eva', 0.8322920799255371)]

Semantic distance between words

In [11]:
def nearest_similarity_cosmul(start1, end1, end2):
    similarities = songs2vec.wv.most_similar_cosmul(
        positive=[end2, start1],
        negative=[end1]
    )
    start2 = similarities[0][0]
    print("{0} es a {1}, lo que {2} es a {3}".format(start1, end1, start2, end2))

In [12]:
nearest_similarity_cosmul("españa", "mar", "tristeza")

españa es a mar, lo que peña es a tristeza


With the word vector embeddings in place, it is now time to calculate the normalised vector sum of each song. This process can take some time since it has to be done for each of 57,000 songs.

In [13]:
print(songs2vec['noche'])
def songVector(row):
    vector_sum = 0
    words = row.lower().split()
    for word in words:
        vector_sum = vector_sum + songs2vec[word]
    vector_sum = vector_sum.reshape(1,-1)
    normalised_vector_sum = sklearn.preprocessing.normalize(vector_sum)
    return normalised_vector_sum


import time
start_time = time.time()

songs['song_vector'] = songs['letra'].apply(songVector)




C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem_

[-5.17074049e-01 -1.29408166e-01 -1.18329832e-02  2.77447611e-01
  2.26320729e-01  1.46254316e-01  9.66741294e-02  1.73064575e-01
  1.67837441e-01 -5.19899428e-01  2.64923144e-02  7.51196742e-01
 -8.49420205e-02  9.56300914e-01  1.36521471e+00 -2.50449568e-01
 -4.00872342e-02 -3.37137848e-01  3.31279576e-01  8.24420869e-01
  5.66081285e-01 -5.58772027e-01  1.55061826e-01 -6.72144234e-01
  8.02902654e-02 -7.50360310e-01  2.09654123e-02  3.12014017e-02
 -1.53752089e-01  5.64178050e-01  2.50010937e-01  2.68684030e-01
 -4.55310851e-01 -7.10956037e-01  2.85135686e-01 -5.27283311e-01
 -1.34423983e+00  4.27128792e-01 -1.79678917e-01 -2.63296887e-02
 -1.42201737e-01  1.97098002e-01 -5.36640584e-01  3.81881863e-01
 -4.53839839e-01 -6.49021706e-04  6.77410662e-01  1.19644545e-01
  5.76114357e-02 -6.44080564e-02]


C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\s

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\s

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\s

...

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\s

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\s

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\s

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\s

C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
C:\Users\merxe\Anaconda3\lib\s

**t-sne and random song selection** 

The songs have 50 dimensions each. Application of t-sne is memory intensive and hence it is slightly easier on the computer to use a random sample of the 57,000 songs.

In [14]:
song_vectors = []
from sklearn.model_selection import train_test_split

train, test = train_test_split(songs, test_size = 0.9)


for song_vector in train['song_vector']:
    song_vectors.append(song_vector)

train.head(10)

,id,artista,cancion,album,letra,anyo,visitas,song_vector
4055,40,Toteking,Intro,Creador series Vol. 1,[Tote King]\nEsto esta dedicao a los papas que...,2004,3344,"[[-0.12101271, -0.025886443, 0.10018119, 0.281..."
2115,25,Ramy,Mi angel,Dejemonos de tonterias,[Estribillo] (x2)\nQuiero tocar las nubes si t...,2007,4979,"[[-0.16397665, -0.073948614, 0.089437895, 0.27..."
1811,51,Eneyser,Quiero sentir el latido,La quinta dimensión,Cuanto más lo pienso menos me importa\ncuanto ...,2009,4887,"[[-0.15115443, -0.06356844, 0.10047601, 0.2769..."
3782,42,Xtragos,Cavernalia,Corazonada,"Hey, estoy desnudo,\nHey, estoy desnudo,\nHey,...",2018,154,"[[-0.1291017, -0.029648423, 0.09768895, 0.2653..."
543,48,Endecah,Bonus track,Tinta negra,[Yesh]\nHabía una tía que me hacía sentir espe...,2011,1959,"[[-0.16050147, -0.07602741, 0.13282019, 0.2579..."
5121,31,Lon MC,Soulitarios (con Elehace),Barbecho,"[EleHache]\nTodo listo ya comienza, los focos ...",2017,234,"[[-0.120170526, -0.056649636, 0.099213615, 0.2..."
4140,15,S&oacute;lo los solo,El chico del 9,Sin álbum,Estoy hablando de mi sitio\npero tú sabes que ...,0,2459,"[[-0.14982526, -0.037458654, 0.11244829, 0.277..."
8429,39,Ceerre,Face off,Cuentos del ánimo,[Estribillo]\nSiempre habrá un rico egoista y ...,2008,1927,"[[-0.15276316, -0.05474045, 0.09027748, 0.2656..."
2749,54,BeMoL,Eskupes pelo,"Aki te rimo, aki te mato",Putas no envidio donde en videos salen mostran...,2007,813,"[[-0.15057342, -0.04106875, 0.09161699, 0.2781..."
8183,13,Xenon,Nunca jamás (con Yarea),Atlántida,El día 23 de Noviembre de 2018 aparece el nuev...,2018,620,"[[-0.14828488, -0.009398861, 0.07090071, 0.294..."


I had a fairly measly 4gb machine and wasn't able to generate a more accurate model. However, one can play around with the number of iterations, learning rate and other factors to fit the model better. If you have too many dimensions (~300+), it might make sense to use PCA first and then t-sne.

In [15]:
X = np.array(song_vectors).reshape((932, 50))

start_time = time.time()
tsne = sklearn.manifold.TSNE(n_components=2, n_iter=250, random_state=0, verbose=2)

all_word_vectors_matrix_2d = tsne.fit_transform(X)

print("--- %s seconds ---" % (time.time() - start_time))

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 932 samples in 0.002s...
[t-SNE] Computed neighbors for 932 samples in 0.120s...
[t-SNE] Computed conditional probabilities for sample 932 / 932
[t-SNE] Mean sigma: 0.028338
[t-SNE] Computed conditional probabilities in 0.094s
[t-SNE] Iteration 50: error = 74.0504150, gradient norm = 0.3228178 (50 iterations in 0.811s)
[t-SNE] Iteration 100: error = 75.3808212, gradient norm = 0.3113391 (50 iterations in 0.655s)
[t-SNE] Iteration 150: error = 75.9347916, gradient norm = 0.2957017 (50 iterations in 0.620s)
[t-SNE] Iteration 200: error = 74.7640762, gradient norm = 0.3189439 (50 iterations in 0.618s)
[t-SNE] Iteration 250: error = 77.1015091, gradient norm = 0.3067400 (50 iterations in 1.014s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 77.101509
[t-SNE] KL divergence after 251 iterations: 17976931348623157081452742373170435679807056752584499659891747680315726078002853876058955863276687817154045895351438246

In [16]:
df=pd.DataFrame(all_word_vectors_matrix_2d,columns=['X','Y'])

df.head(10)

train.head()

df.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)

Joining two dataframes to obtain each song's corresponding X,Y co-ordinate.

In [17]:
two_dimensional_songs = pd.concat([train, df], axis=1)

two_dimensional_songs.head()

,id,artista,cancion,album,letra,anyo,visitas,song_vector,X,Y
0,40,Toteking,Intro,Creador series Vol. 1,[Tote King]\nEsto esta dedicao a los papas que...,2004,3344,"[[-0.12101271, -0.025886443, 0.10018119, 0.281...",0.345093,-0.359623
1,25,Ramy,Mi angel,Dejemonos de tonterias,[Estribillo] (x2)\nQuiero tocar las nubes si t...,2007,4979,"[[-0.16397665, -0.073948614, 0.089437895, 0.27...",-0.309699,0.326080
2,51,Eneyser,Quiero sentir el latido,La quinta dimensión,Cuanto más lo pienso menos me importa\ncuanto ...,2009,4887,"[[-0.15115443, -0.06356844, 0.10047601, 0.2769...",0.038432,-0.052183
3,42,Xtragos,Cavernalia,Corazonada,"Hey, estoy desnudo,\nHey, estoy desnudo,\nHey,...",2018,154,"[[-0.1291017, -0.029648423, 0.09768895, 0.2653...",0.026936,0.034817
4,48,Endecah,Bonus track,Tinta negra,[Yesh]\nHabía una tía que me hacía sentir espe...,2011,1959,"[[-0.16050147, -0.07602741, 0.13282019, 0.2579...",-0.274762,0.349034


**Plotting the results**

Using plotly, I plotted the results so that it becomes easier to explore similar songs based on their colors and clusters.

In [18]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

import plotly.graph_objs as go

trace1 = go.Scatter(
    y = two_dimensional_songs['Y'],
    x = two_dimensional_songs['X'],
    text = two_dimensional_songs['artista'],
    mode='markers',
    marker=dict(
        size= 5,#'7',
        color = np.random.randn(5717), #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
)
data = [trace1]

iplot(data)

In [19]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure(data=go.Scatter(
    y = two_dimensional_songs['Y'],
    x = two_dimensional_songs['X'],
    text = two_dimensional_songs['artista']+ "_"+two_dimensional_songs['cancion'] ,
    mode='markers',
    marker=dict(
        size= 10,#'7',
        color = np.random.randn(5717), #set color equal to a variable
        colorscale='Viridis',
        showscale=True
    )
))

fig.show()

In [20]:
## LOOK FOR COMMON SONGS AND ANALYZE THE TEXT

In [21]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure(data=go.Scatter(
    y = np.random.randn(500),
    mode='markers',
    marker=dict(
        size=16,
        color=np.random.randn(500), #set color equal to a variable
        colorscale='Viridis', # one of plotly colorscales
        showscale=True
    )
))

fig.show()

In [22]:
import plotly.express as px
df = px.data.iris()
fig = px.scatter_3d(df, x='sepal_length', y='sepal_width', z='petal_width',
              color='species')
fig.show()